<a href="https://colab.research.google.com/github/sakshamkumar1/tensorflow-notebooks/blob/main/08_introduction_to_nlp_in_tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in TensorFlow

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-7153f991-148e-fecb-a03b-db294d84857e)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2023-05-15 14:38:33--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-05-15 14:38:33 (70.7 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2023-05-15 14:38:39--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.215.128, 74.125.134.128, 173.194.212.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.215.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.01s   

2023-05-15 14:38:39 (53.2 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

In [4]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(train_df), len(test_df)

(7613, 3263)

In [9]:
import random
random_index = random.randint(0, len(train_df)-5)
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster)")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster)
Text:
I wish that the earth sea and sky up above
would send me someone to lava????

---

Target: 0 (not real disaster)
Text:
Flat out bomb by @FlavaFraz21 #whatcanthedo

---

Target: 1 (real disaster)
Text:
Russian 'food crematoria' provoke outrage amid crisis famine memories: MOSCOW (Reuters) - Russian government ... http://t.co/Mphog0QDDN

---

Target: 1 (real disaster)
Text:
Mexico: construction of bridge collapse killsåÊone http://t.co/I2C00FcOwb http://t.co/jAAgcFaRTW

---

Target: 0 (not real disaster)
Text:
A grade in Black Horse Famine[MEGA]. Score 0840728
http://t.co/pdiit0AF3Q
#Dynamix http://t.co/ZQ5KDOx7BY

---



### Split data into training and validation sets

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
train_df.iloc[:2].to_numpy()

array([[1, nan, nan,
        'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
        1],
       [4, nan, nan, 'Forest fire near La Ronge Sask. Canada', 1]],
      dtype=object)

In [12]:
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [13]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers

### Text vectorization (tokenization)

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

text_vectorizer = TextVectorization(max_tokens=None,
                                    standardize="lower_and_strip_punctuation",
                                    split="whitespace",
                                    ngrams=None,
                                    output_mode="int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=False)

In [17]:
len(train_sentences[0].split())

7

In [18]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

In [19]:
text_vectorizer.adapt(train_sentences)

In [20]:
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 @james_justus *returns her*

Relax. You know I always return her so no need to panic. I just gave her some of those corn flakes. :P        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[   1, 3470,   81, 3480,   12,  106,    8,  262, 1609,   81,   28,
          40,  162,    5,  297]])>

In [22]:
text_vectorizer(random_sentence)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([   1, 3470,   81, 3480,   12,  106,    8,  262, 1609,   81,   28,
         40,  162,    5,  297])>

In [23]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
print(f"Most common words in vocab: {top_5_words}")
print(f"Least common words in vocab:{bottom_5_words}")

Most common words in vocab: ['', '[UNK]', 'the', 'a', 'in']
Least common words in vocab:['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


In [24]:
len(words_in_vocab)

10000

### Creating an Embedding using an Embedding Layer

In [25]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length=max_length)
embedding

In [26]:
random_sentence = random.choice(train_sentences)
print(f"Original test:\n{random_sentence}\
        \n\nEmbedded version:")

sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original test:
RT AbbsWinston: #Zionist #Terrorist demolish 18 #Palestinian structures in Jordan Valley http://t.co/rg3BndKXjX
Û_ http://t.co/Bq90pfzMrP        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.03467616, -0.0342643 ,  0.00630742, ...,  0.04154605,
         -0.03660672,  0.0375814 ],
        [-0.04743347,  0.03811331, -0.04269478, ..., -0.03459286,
         -0.03361257, -0.03828849],
        [ 0.04001207,  0.01410829,  0.01271683, ..., -0.04297819,
          0.02652775, -0.02244201],
        ...,
        [ 0.01200981,  0.00329579, -0.04990851, ..., -0.01494585,
         -0.02158717,  0.02822823],
        [ 0.02565707,  0.042218  , -0.01500001, ...,  0.01694337,
          0.01047481, -0.01948756],
        [-0.01280019, -0.03466039, -0.04763948, ..., -0.04637271,
          0.04710735,  0.04576756]]], dtype=float32)>

In [27]:
sample_embed[0][0]

<tf.Tensor: shape=(128,), dtype=float32, numpy=
array([ 0.03467616, -0.0342643 ,  0.00630742, -0.00067813, -0.0475413 ,
        0.00566807,  0.00646694,  0.02729925, -0.00518559, -0.04780669,
        0.01989975, -0.01886178, -0.02136475,  0.00220527, -0.01119126,
        0.02594837, -0.02770184,  0.04361876,  0.0137053 ,  0.01974102,
        0.03815006, -0.02785507, -0.02859534, -0.01883348,  0.04246599,
       -0.04212048, -0.02714743,  0.01586911, -0.02437356, -0.02428528,
        0.0388646 ,  0.03438887, -0.00166195,  0.03501675,  0.01174311,
        0.01859976, -0.00848448,  0.00538553,  0.02135989, -0.04774009,
       -0.0470576 ,  0.03751416,  0.00785808, -0.03959067, -0.04766626,
        0.01574694, -0.04350125,  0.03523064, -0.03467352,  0.04656443,
        0.00126072, -0.03730272,  0.03001598, -0.00420235,  0.02634383,
        0.02439487,  0.00388364,  0.03597537,  0.01489666, -0.03642445,
        0.0438009 ,  0.04079732,  0.00648239,  0.03267925,  0.01691613,
        0.017708

### Model 0: Getting a Baseline

In [28]:
# from sklearn.naive_bayes import GaussianNB

# gnb = GaussianNB()
# gnb.fit(train_sentences, train_labels)
# gnb.score(val_sentences, val_labels)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [30]:
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [31]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

## Creating an evaluation function for our model experiments

In [32]:
# Function to evaluate: accuracy, precision, recall, F1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [33]:
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

## Model 1: A simple dense model (feed forward neural network)

In [34]:
from helper_functions import create_tensorboard_callback

SAVE_DIR = "model_logs"

In [35]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [36]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [37]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_1 = model_1.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230515-143848
Epoch 1/5
215/215 [==============================] - 23s 65ms/step - loss: 0.6102 - accuracy: 0.6903 - val_loss: 0.5382 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.4402 - accuracy: 0.8192 - val_loss: 0.4708 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3462 - accuracy: 0.8589 - val_loss: 0.4588 - val_accuracy: 0.7979
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.2832 - accuracy: 0.8920 - val_loss: 0.4665 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2360 - accuracy: 0.9126 - val_loss: 0.4761 - val_accuracy: 0.7835


In [38]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4761 - accuracy: 0.7835


[0.47606560587882996, 0.7834645509719849]

In [39]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[0.3883664 ],
       [0.8073469 ],
       [0.9972051 ],
       [0.13460527],
       [0.10733739],
       [0.9390411 ],
       [0.9215221 ],
       [0.9930078 ],
       [0.97013783],
       [0.30648583]], dtype=float32)

In [40]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [41]:
val_labels[:10]

array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0])

In [42]:
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.34645669291339,
 'precision': 0.7861644127328657,
 'recall': 0.7834645669291339,
 'f1': 0.7811856084066165}

In [43]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [44]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Model 2: LSTM

In [45]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
x = layers.LSTM(64, activation="tanh", return_sequences=True)(x)
print(f"After LSTM cell: {x.shape}")
x = layers.LSTM(64, activation="tanh")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM")

After embedding: (None, 15, 128)
After LSTM cell: (None, 15, 64)


In [46]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_2 = model_2.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230515-143933
Epoch 1/5
215/215 [==============================] - 17s 50ms/step - loss: 0.2174 - accuracy: 0.9266 - val_loss: 0.5488 - val_accuracy: 0.7848
Epoch 2/5
215/215 [==============================] - 5s 21ms/step - loss: 0.1567 - accuracy: 0.9400 - val_loss: 0.5980 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 4s 16ms/step - loss: 0.1271 - accuracy: 0.9521 - val_loss: 0.6468 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.1052 - accuracy: 0.9599 - val_loss: 0.7512 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0819 - accuracy: 0.9661 - val_loss: 0.9421 - val_accuracy: 0.7756


In [47]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs.shape, model_2_pred_probs[:10]

24/24 [==============================] - 1s 3ms/step


((762, 1),
 array([[5.4585082e-03],
        [8.4537339e-01],
        [9.9995279e-01],
        [1.8641962e-01],
        [1.6337509e-04],
        [9.9990809e-01],
        [9.4667947e-01],
        [9.9995458e-01],
        [9.9994886e-01],
        [3.5374266e-01]], dtype=float32))

In [48]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [49]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_preds)
model_2_results

{'accuracy': 77.55905511811024,
 'precision': 0.7759863909628747,
 'recall': 0.7755905511811023,
 'f1': 0.7743062301518678}

## Model 3: GRU

In [50]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
x = layers.GRU(64, activation="tanh", return_sequences=True)(x)
print(f"After GRU cell: {x.shape}")
x = layers.GRU(64, activation="tanh")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs, outputs, name="model_3_GRU")

After embedding: (None, 15, 128)
After GRU cell: (None, 15, 64)


In [51]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_3 = model_3.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230515-144005
Epoch 1/5
215/215 [==============================] - 16s 58ms/step - loss: 0.1473 - accuracy: 0.9426 - val_loss: 0.7303 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0843 - accuracy: 0.9680 - val_loss: 0.7526 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0719 - accuracy: 0.9718 - val_loss: 1.0786 - val_accuracy: 0.7795
Epoch 4/5
215/215 [==============================] - 3s 13ms/step - loss: 0.0602 - accuracy: 0.9750 - val_loss: 1.0028 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0585 - accuracy: 0.9765 - val_loss: 1.2835 - val_accuracy: 0.7756


## Model 4: Bidirectional RNN Model

In [52]:
from tensorflow.keras import layers
from tensorflow.keras.layers import LSTM
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(f"After embedding: {x.shape}")
x = layers.Bidirectional(LSTM(64, activation="tanh", return_sequences=True))(x)
print(f"After Bidirectional LSTM cell: {x.shape}")
x = layers.LSTM(64, activation="tanh")(x)
print(x.shape)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs, outputs, name="model_4_Bidirectional_LSTM")

After embedding: (None, 15, 128)
After Bidirectional LSTM cell: (None, 15, 128)
(None, 64)


In [53]:
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_4 = model_4.fit(train_sentences,
                        train_labels,
                        epochs=5,
                        validation_data=(val_sentences, val_labels),
                        callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                               experiment_name="model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230515-144033
Epoch 1/5
215/215 [==============================] - 19s 59ms/step - loss: 0.1015 - accuracy: 0.9672 - val_loss: 1.0870 - val_accuracy: 0.7782
Epoch 2/5
215/215 [==============================] - 4s 16ms/step - loss: 0.0500 - accuracy: 0.9778 - val_loss: 1.1352 - val_accuracy: 0.7756
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.0487 - accuracy: 0.9788 - val_loss: 1.3189 - val_accuracy: 0.7743
Epoch 4/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0473 - accuracy: 0.9794 - val_loss: 1.3683 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0417 - accuracy: 0.9813 - val_loss: 1.3432 - val_accuracy: 0.7756


In [54]:
model_4.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 6ms/step - loss: 1.3432 - accuracy: 0.7756


[1.34319007396698, 0.7755905389785767]

In [55]:
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10], val_labels[:10]

24/24 [==============================] - 1s 4ms/step


(array([[7.5243053e-04],
        [7.3841858e-01],
        [9.9986327e-01],
        [1.6002086e-01],
        [1.9348528e-04],
        [9.9981946e-01],
        [1.4264956e-01],
        [9.9988711e-01],
        [9.9987745e-01],
        [8.7754685e-01]], dtype=float32),
 array([0, 0, 1, 1, 1, 1, 1, 1, 1, 0]))

In [56]:
model_4_pred_probs.shape

(762, 1)

In [57]:
val_sentences.shape

(762,)

In [58]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [59]:
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 77.55905511811024,
 'precision': 0.7797991425654591,
 'recall': 0.7755905511811023,
 'f1': 0.7725109164504043}

## Model 5: Conv1D

In [65]:
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d_layer = layers.Conv1D(filters=32,
                              kernel_size=5,
                              activation="relu")
conv_1d_output = conv_1d_layer(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)
print(f"Embedding output shape: {embedding_test.shape}")
print(f"Conv1D output shape: {conv_1d_output.shape}")
print(f"Max pool  output shape: {max_pool_output.shape}")

Embedding output shape: (1, 15, 128)
Conv1D output shape: (1, 11, 32)
Max pool  output shape: (1, 32)


In [62]:
print(f"Embedding output: {embedding_test}")
print(f"Conv1D output: {conv_1d_output}")
print(f"Max pool output: {max_pool_output}")

Embedding output: [[[-0.02147716 -0.04465713 -0.06838858 ... -0.06160404  0.06256708
    0.02489355]
  [-0.01901681  0.02633145  0.00685457 ... -0.02148594 -0.02558788
   -0.0108084 ]
  [ 0.01643557 -0.020492    0.02350922 ... -0.02718012 -0.02293899
   -0.06714181]
  ...
  [ 0.00379954 -0.00611658 -0.03328801 ... -0.02522699  0.00949321
   -0.01342106]
  [ 0.00379954 -0.00611658 -0.03328801 ... -0.02522699  0.00949321
   -0.01342106]
  [ 0.00379954 -0.00611658 -0.03328801 ... -0.02522699  0.00949321
   -0.01342106]]]
Conv1D output: [[[0.         0.         0.         0.         0.0070861  0.03534135
   0.07491001 0.07947917 0.         0.         0.         0.
   0.08724189 0.08133584 0.03953607 0.         0.         0.
   0.03467459 0.05405578 0.         0.03119896 0.01546054 0.0538167
   0.         0.         0.12364988 0.01917772 0.03073546 0.00043371
   0.         0.        ]
  [0.0537134  0.06869896 0.03028581 0.00436804 0.         0.06128523
   0.         0.0068702  0.         0.

In [66]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=32, kernel_size=5, activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation="sigmoid", name="output_layer")(x)
model_5 = tf.keras.Model(inputs, outputs, name="model_5_conv1d")

model_5.summary()

Model: "model_5_conv1d"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_3 (Conv1D)           (None, 11, 32)            20512     
                                                                 
 global_max_pooling1d_2 (Glo  (None, 32)               0         
 balMaxPooling1D)                                                
                                                                 
 output_layer (Dense)        (None, 1)              

In [67]:
BATCH_SIZE = 32

In [68]:
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

history_model_5 = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              batch_size=BATCH_SIZE,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_5_conv1d")])

Saving TensorBoard log files to: model_logs/model_5_conv1d/20230515-153506
Epoch 1/5
215/215 [==============================] - 15s 57ms/step - loss: 0.1399 - accuracy: 0.9548 - val_loss: 0.8168 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0767 - accuracy: 0.9737 - val_loss: 0.9542 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0618 - accuracy: 0.9766 - val_loss: 1.0462 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0559 - accuracy: 0.9788 - val_loss: 1.1492 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0513 - accuracy: 0.9778 - val_loss: 1.2181 - val_accuracy: 0.7598


In [69]:
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:5], model_5_pred_probs.shape

24/24 [==============================] - 0s 2ms/step


(array([[2.3431164e-01],
        [7.6052517e-01],
        [9.9993253e-01],
        [8.4624447e-02],
        [3.3369622e-07]], dtype=float32),
 (762, 1))

In [70]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [72]:
model_5_results = calculate_results(y_true=val_labels,
                                  y_pred=model_5_preds)
model_5_results

{'accuracy': 75.98425196850394,
 'precision': 0.7604578907479682,
 'recall': 0.7598425196850394,
 'f1': 0.758116023760764}